# Model Metrics Verification

### Due to the private non-exportable nature of our dataset, this notebook is meant to verify the metrics shown in our report. Here you will see how each model is loaded from a saved state and evaluated on the test set. The importatnt verification cells appear halfway through the notebook

In [1]:
!nvidia-smi

Fri Dec  8 13:50:11 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1080 Ti     Off | 00000000:04:00.0 Off |                  N/A |
| 23%   36C    P0              60W / 250W |      0MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import sys
sys.path.append('../')

In [3]:
import time
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, roc_auc_score
import matplotlib.pyplot as plt
import math
import numpy as np
import seaborn as sns
import os
import tensorflow as tf
import sys
import tensorflow_addons as tfa
from sklearn.metrics import roc_curve, auc
import pandas as pd
from sklearn.model_selection import StratifiedGroupKFold
import numpy as np
import nibabel as nib
import tensorflow as tf
from scipy.ndimage import zoom
import matplotlib.gridspec as gridspec
import model.TCNN, model.VGG.tiny_vgg, model.Densenet.Densenet3D, model.ResNet.ResNet_Conv

2023-12-08 13:50:16.851936: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [4]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices), ", GPU ID: ", "0")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1 , GPU ID:  0


2023-12-08 13:50:24.844198: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-12-08 13:50:24.846450: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-12-08 13:50:29.798838: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:04:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2023-12-08 13:50:29.798900: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-12-08 13:50:29.803626: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-12-08 13:50:29.803680: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.

# Load Dataset From Server

In [5]:
class Data:
    def __init__(self, args):
        self.args = args
        self.train_pathway = ""
        self.test_pathway = ""
        self.pathway = "/lfs1/pjtoral/cognitive-decline/scripts/data/revised/standardized/mci_included"
        self.dof = "9DOF"
        self.target_column = "MMSE"
        self.batch_size = 1
        # self.transformation = self.args.transformation
        # self.sample_weight = self.args.sample_weight
        # if self.sample_weight == "dense_weight":
        #     self.dense_weight_alpha = experiment_config["alpha"]

        self.train_df = pd.DataFrame()
        self.validation_df = pd.DataFrame()
        self.test_df = pd.DataFrame()
        self.train_batch = pd.DataFrame()
        self.validation_batch = pd.DataFrame()
        self.test_batch = pd.DataFrame()

        self.set_dataframes()
        self.set_data_generators()

    def set_dataframes(self):
        df_train_ADNI1 = pd.read_csv(self.pathway + "/train_ADNI1_" + self.dof + ".csv")
        df_train_ADNI2 = pd.read_csv(self.pathway + "/train_ADNI2_" + self.dof + ".csv")
        df_train_ADNI3 = pd.read_csv(self.pathway + "/train_ADNI3_" + self.dof + ".csv")
        df_test_ADNI1 = pd.read_csv(self.pathway + "/test_ADNI1_" + self.dof + ".csv")
        df_test_ADNI2 = pd.read_csv(self.pathway + "/test_ADNI2_" + self.dof + ".csv")
        df_test_ADNI3 = pd.read_csv(self.pathway + "/test_ADNI3_" + self.dof + ".csv")
        df_train = pd.concat([df_train_ADNI1, df_train_ADNI2, df_train_ADNI3], ignore_index=True).reset_index(drop=True)
        df_test = pd.concat([df_test_ADNI1, df_test_ADNI2, df_test_ADNI3], ignore_index=True).reset_index(drop=True)

        # df_train = pd.read_csv(self.train_pathway)
        # df_test = pd.read_csv(self.test_pathway)

        df_train.dropna(subset=[self.target_column], inplace=True)
        df_test.dropna(subset=[self.target_column], inplace=True)
        self.test_df = df_test
        self.split_dataframes(df_train)

    def split_dataframes(self, df_train):
        df_train.reset_index(inplace=True)
        df_validation = pd.DataFrame()
        # ADNI
        df_train["subj_id"] = ["_".join(x.split("/")[-1].split("_")[:3]) for x in df_train['volume']]
        sgkf = StratifiedGroupKFold(n_splits=2, shuffle=True, random_state=7)
        (train_idxs, validation_idxs) = next(
            sgkf.split(df_train.drop(columns=["label"]), df_train["label"], groups=df_train["subj_id"]))
        df_train_ = df_train.iloc[train_idxs]
        df_validation = df_train.iloc[validation_idxs]

        sgkf.split(df_validation.drop(columns=["label"]), df_validation["label"], groups=df_validation["subj_id"])
        (train_addition_idx, true_validation_idxs) = next(
            sgkf.split(df_validation.drop(columns=["label"]), df_validation["label"], groups=df_validation["subj_id"]))
        df_train_ = pd.concat([df_train_, df_validation.iloc[train_addition_idx]], ignore_index=True)
        df_validation = df_validation.iloc[true_validation_idxs]

        df_train = df_train_.copy()
        self.train_df = df_train
        self.validation_df = df_validation

    def set_data_generators(self):
        train_x = self.train_df["volume"].to_numpy()
        train_y = self.train_df[self.target_column].to_numpy().astype(np.float32)
        validate_x = self.validation_df["volume"].to_numpy()
        validate_y = self.validation_df[self.target_column].to_numpy().astype(np.float32)
        test_x = self.test_df["volume"].to_numpy()
        test_y = self.test_df[self.target_column].to_numpy().astype(np.float32)


        # if self.sample_weight == "dense_weight":
        #     dw = DenseWeight(alpha=self.dense_weight_alpha)
        #     sample_weights = dw.fit(train_y)
        #     self.train_batch = self.DataGenerator(train_x, train_y, self.batch_size, sample_weights)

        self.train_batch = self.DataGenerator(train_x, train_y, self.batch_size)
        self.validation_batch = self.DataGenerator(validate_x, validate_y, self.batch_size)
        self.test_batch = self.DataGenerator(test_x, test_y, self.batch_size)

    class DataGenerator(tf.keras.utils.Sequence):
        def read_scan(self, path):
            scan = nib.load(path)
            original_volume = scan.get_fdata()
            original_volume_normalized = self.normalize(original_volume)
            resized_volume = self.resize(original_volume_normalized)
            return tf.expand_dims(resized_volume, axis=3)

        def normalize(self, volume):
            min = np.amax(volume)
            max = np.amin(volume)
            volume = (volume - min) / (max - min)
            volume = volume.astype("float32")
            return volume

        def resize(self, original_volume, w=96, h=96, d=96):
            zoom_factors = (w / original_volume.shape[0], h / original_volume.shape[1], d / original_volume.shape[2])
            resized_volume = zoom(original_volume, zoom_factors)
            resized_volume_normalized = self.normalize(resized_volume)
            return resized_volume_normalized

        def display(self):
            path = self.image_filenames[0]
            scan = nib.load(path)
            print(path)
            original_volume = scan.get_fdata()
            original_volume_normalized = self.normalize(original_volume)
            resized_volume = self.resize(original_volume_normalized)
        
            # Get the middle slice of the original image
            original_slice = original_volume[:, :, original_volume.shape[2] // 2]
            # Get the middle slice of the resized image
            resized_slice = resized_volume[:, :, resized_volume.shape[2] // 2]
            #
            # Plot the slices
            plt.figure(figsize=(12, 6))
            plt.imshow(original_slice, cmap='gray')
            plt.title('MMSE Score: '+str(self.labels[0]))
            plt.savefig('mri.png')
#             plt.show()
            plt.imshow(resized_slice, cmap='gray')
            plt.title('Resized Image ' + str(resized_volume.shape),)
#             plt.show()

        def __init__(self, image_filenames, labels, batch_size, sample_weights=None):
            self.image_filenames = image_filenames
            self.labels = labels
            self.batch_size = batch_size

        def __len__(self):
            return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)

        def __getitem__(self, idx):
            batch_x = self.image_filenames[idx * self.batch_size: (idx + 1) * self.batch_size]
            batch_y = self.labels[idx * self.batch_size: (idx + 1) * self.batch_size]
            return (np.asarray([self.read_scan(path) for path in batch_x]), np.array(batch_y))

### Retrieve Test Set from the Batch Generator

In [6]:
args = {"drop_out":0.0}
data = Data(args)
test_batch = data.test_batch

train_mean = np.mean(data.train_df["MMSE"])
train_std = np.std(data.train_df["MMSE"])

# Load All Saved Models

In [7]:
print(args)
resnet_baseline = model.ResNet.ResNet_Conv.RESNET3D(args, train_mean, train_std).get_model(False)
resnet_improvement = model.ResNet.ResNet_Conv.RESNET3D(args, train_mean, train_std).get_model(False)
vgg_baseline = model.VGG.tiny_vgg.TinyVGG(args, train_mean, train_std).get_model(False)
vgg_improvement = model.VGG.tiny_vgg.TinyVGG(args, train_mean, train_std).get_model(False)
tcnn_baseline = model.TCNN.TCNN(args, train_mean, train_std).get_model(False)
tcnn_improvement = model.TCNN.TCNN(args, train_mean, train_std).get_model(False)
densenet_baseline = model.Densenet.Densenet3D.DenseNet3D(args, train_mean, train_std, depth=121, nb_dense_block=4,
                                        growth_rate=32,
                                        nb_filter=64, nb_layers_per_block=[6, 12, 24, 16],
                                        bottleneck=False, reduction=0.0,
                                        dropout_rate=0.0, weight_decay=1e-4,
                                        subsample_initial_block=True, include_top=False,
                                        input_shape=(96, 96, 96, 1), pooling="max")
densenet_improvement = model.Densenet.Densenet3D.DenseNet3D(args, train_mean, train_std, depth=121, nb_dense_block=4,
                                        growth_rate=32,
                                        nb_filter=64, nb_layers_per_block=[6, 12, 24, 16],
                                        bottleneck=False, reduction=0.0,
                                        dropout_rate=0.0, weight_decay=1e-4,
                                        subsample_initial_block=True, include_top=False,
                                        input_shape=(96, 96, 96, 1), pooling="max")

optimizer = tfa.optimizers.AdamW(learning_rate=0.005,
                                                  weight_decay=0.0001) 
loss_fn = tf.keras.losses.MeanSquaredError(
                reduction=tf.keras.losses.Reduction.AUTO,
                name='mean_squared_error'
            )
metrics = ["mse", "mae"]    

{'drop_out': 0.0}
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2023-12-08 13:50:30.433535: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-08 13:50:30.439683: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-12-08 13:50:30.440199: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:04:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2023-12-08 13:50:30.440261: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-12-08 13:50:30.440293: I tensorflow/stream_executor/platform

# Retrieve Predictions on Test Set from Saved Models

In [8]:
resnet_baseline.set_weights(tf.keras.models.load_model("../output/resnet_baseline__2023-11-27_12:37:39/save", compile=False).get_weights())
resnet_improvement.set_weights(tf.keras.models.load_model("../output/resnet_tl_improvement__2023-12-03_13:05:18/save", compile=False).get_weights())
resnet_baseline.compile(
            optimizer=optimizer,
            loss=loss_fn,
            metrics=metrics
        )
resnet_improvement.compile(
            optimizer=optimizer,
            loss=loss_fn,
            metrics=metrics
        )

start = time.time()
resnet_baseline_predictions = [p[0] for p in resnet_baseline.predict(test_batch)]
end = time.time()
print("Resnet Cold-Start Inference Time: "+ str(end-start) + " seconds")


start = time.time()
resnet_improvement_predictions = [p[0] for p in resnet_improvement.predict(test_batch)]
end = time.time()
print("Resnet Fine-Tuned Inference Time: "+ str(end-start) + " seconds")




/nas/home/pjtoral/.conda/envs/pjt/lib/python3.7/site-packages/ipykernel_launcher.py:132: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
2023-12-08 13:50:37.713695: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-12-08 13:50:37.714429: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2199895000 Hz
2023-12-08 13:50:38.170539: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
202

Resnet Cold-Start Inference Time: 1041.3665878772736 seconds
Resnet Fine-Tuned Inference Time: 1026.49955868721 seconds


In [9]:
vgg_baseline.set_weights(tf.keras.models.load_model("../output/vgg_baseline_no_wd__2023-11-24_11:26:27/save", compile=False).get_weights())
vgg_improvement.set_weights(tf.keras.models.load_model("../output/tiny_vgg_tl_improvement__2023-12-06_13:05:25/save", compile=False).get_weights())
vgg_baseline.compile(
            optimizer=optimizer,
            loss=loss_fn,
            metrics=metrics
        )
vgg_improvement.compile(
            optimizer=optimizer,
            loss=loss_fn,
            metrics=metrics
        )

start = time.time()
vgg_baseline_predictions = [p[0] for p in vgg_baseline.predict(test_batch)]
end = time.time()
print("Tiny VGG Cold-Start Inference Time: "+ str(end-start) + " seconds")


start = time.time()
vgg_improvement_predictions = [p[0] for p in vgg_improvement.predict(test_batch)]
end = time.time()
print("Tiny VGG Fine-Tuned Inference Time: "+ str(end-start) + " seconds")

/nas/home/pjtoral/.conda/envs/pjt/lib/python3.7/site-packages/ipykernel_launcher.py:132: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Tiny VGG Cold-Start Inference Time: 1003.48930311203 seconds
Tiny VGG Fine-Tuned Inference Time: 1005.3480904102325 seconds


In [10]:
tcnn_baseline.set_weights(tf.keras.models.load_model("../output/tcnn_baseline_no_wd__2023-11-24_11:27:16/save", compile=False).get_weights())
tcnn_improvement.set_weights(tf.keras.models.load_model("../output/tcnn_tl_improvement__2023-11-30_13:40:13/save", compile=False).get_weights())
tcnn_baseline.compile(
            optimizer=optimizer,
            loss=loss_fn,
            metrics=metrics
        )
tcnn_improvement.compile(
            optimizer=optimizer,
            loss=loss_fn,
            metrics=metrics
        )

start = time.time()
tcnn_baseline_predictions = [p[0] for p in tcnn_baseline.predict(test_batch)]
end = time.time()
print("TCNN Cold-Start Inference Time: "+ str(end-start) + " seconds")


start = time.time()
tcnn_improvement_predictions = [p[0] for p in tcnn_improvement.predict(test_batch)]
end = time.time()
print("TCNN Fine-Tuned Inference Time: "+ str(end-start) + " seconds")

/nas/home/pjtoral/.conda/envs/pjt/lib/python3.7/site-packages/ipykernel_launcher.py:132: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


TCNN Cold-Start Inference Time: 1013.9330520629883 seconds
TCNN Fine-Tuned Inference Time: 1024.0299062728882 seconds


In [11]:
densenet_baseline.set_weights(tf.keras.models.load_model("../output/densenet_baseline_no_wd__2023-11-24_11:26:17/save", compile=False).get_weights())
densenet_improvement.set_weights(tf.keras.models.load_model("../output/densenet_tl_improvement__2023-11-30_13:53:48/save", compile=False).get_weights())
densenet_baseline.compile(
            optimizer=optimizer,
            loss=loss_fn,
            metrics=metrics
        )
densenet_improvement.compile(
            optimizer=optimizer,
            loss=loss_fn,
            metrics=metrics
        )

start = time.time()
densenet_baseline_predictions = [p[0] for p in densenet_baseline.predict(test_batch)]
end = time.time()
print("Densenet Cold-Start Inference Time: "+ str(end-start) + " seconds")


start = time.time()
densenet_improvement_predictions = [p[0] for p in densenet_improvement.predict(test_batch)]
end = time.time()
print("Densenet Fine-Tuned Inference Time: "+ str(end-start) + " seconds")

/nas/home/pjtoral/.conda/envs/pjt/lib/python3.7/site-packages/ipykernel_launcher.py:132: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Densenet Cold-Start Inference Time: 1050.1901426315308 seconds
Densenet Fine-Tuned Inference Time: 1002.3589644432068 seconds


# Results 
## Resnet

In [12]:
test_set = data.test_df["MMSE"].values
print("Resnet Cold-Start")
print("MSE:",round(mean_squared_error(test_set,resnet_baseline_predictions),3))
print("MAE: ",round(mean_absolute_error(test_set,resnet_baseline_predictions),3))
print("R2: ",round(r2_score(test_set,resnet_baseline_predictions),3))

print("\nResnet Fine-Tuned")
print("MSE:",round(mean_squared_error(test_set,resnet_improvement_predictions),3))
print("MAE: ",round(mean_absolute_error(test_set,resnet_improvement_predictions),3))
print("R2: ",round(r2_score(test_set,resnet_improvement_predictions),3))

Resnet Cold-Start
MSE: 8.633
MAE:  2.106
R2:  0.353

Resnet Fine-Tuned
MSE: 8.341
MAE:  2.1
R2:  0.375


## TCNN

In [13]:

print("TCNN Cold-Start")
print("MSE:",round(mean_squared_error(test_set,tcnn_baseline_predictions),3))
print("MAE: ",round(mean_absolute_error(test_set,tcnn_baseline_predictions),3))
print("R2: ",round(r2_score(test_set,tcnn_baseline_predictions),3))

print("\nTCNN Fine-Tuned")
print("MSE:",round(mean_squared_error(test_set,tcnn_improvement_predictions),3))
print("MAE: ",round(mean_absolute_error(test_set,tcnn_improvement_predictions),3))
print("R2: ",round(r2_score(test_set,tcnn_improvement_predictions),3))

TCNN Cold-Start
MSE: 9.333
MAE:  2.219
R2:  0.301

TCNN Fine-Tuned
MSE: 8.871
MAE:  2.138
R2:  0.335


## Tiny VGG

In [14]:
print("Tiny VGG Cold-Start")
print("MSE:",round(mean_squared_error(test_set,vgg_baseline_predictions),3))
print("MAE: ",round(mean_absolute_error(test_set,vgg_baseline_predictions),3))
print("R2: ",round(r2_score(test_set,vgg_baseline_predictions),3))

print("\nTiny VGG Fine-Tuned")
print("MSE:",round(mean_squared_error(test_set,vgg_improvement_predictions),3))
print("MAE: ",round(mean_absolute_error(test_set,vgg_improvement_predictions),3))
print("R2: ",round(r2_score(test_set,vgg_improvement_predictions),3))

Tiny VGG Cold-Start
MSE: 8.929
MAE:  2.118
R2:  0.331

Tiny VGG Fine-Tuned
MSE: 8.565
MAE:  2.09
R2:  0.358


## Densenet

In [15]:
print("Densenet Cold-Start")
print("MSE:",round(mean_squared_error(test_set,densenet_baseline_predictions),3))
print("MAE: ",round(mean_absolute_error(test_set,densenet_baseline_predictions),3))
print("R2: ",round(r2_score(test_set,densenet_baseline_predictions),3))

print("\nDensenetFine-Tuned")
print("MSE:",round(mean_squared_error(test_set,densenet_improvement_predictions),3))
print("MAE: ",round(mean_absolute_error(test_set,densenet_improvement_predictions),3))
print("R2: ",round(r2_score(test_set,densenet_improvement_predictions),3))

Densenet Cold-Start
MSE: 10.652
MAE:  2.244
R2:  0.202

DensenetFine-Tuned
MSE: 9.394
MAE:  2.091
R2:  0.296
